In [6]:
from langchain.chat_models.openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

In [7]:
# Custom imports:
from content_collection import collect_serp_data_and_extract_text_from_webpages
from custom_summarize_chain import create_all_summaries, DocumentSummary
from expert_interview_chain import InterviewChain

## Topic Research:

In [8]:
%pip install google-search-results pandas html2text pytest-playwright chromadb nest_asyncio --quiet

Note: you may need to restart the kernel to use updated packages.


In [9]:
!playwright install

In [10]:
import nest_asyncio
nest_asyncio.apply()

In [12]:
# Constant variables:
TOPIC = "Neural networks"
os.environ["SERPAPI_API_KEY"] = ""
os.environ["STABILITY_API_KEY"] = ""

In [13]:
# Extract content from webpages into LangChain documents:
text_documents = await collect_serp_data_and_extract_text_from_webpages(topic=TOPIC)

In [14]:
# LLM, text splitter + parser:
llm = ChatOpenAI(temperature=0)
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1500, chunk_overlap=400
)
parser = PydanticOutputParser(pydantic_object=DocumentSummary)

In [15]:
summaries = await create_all_summaries(text_documents, parser, llm, text_splitter)

Summarizing the data!
Summarizing the data!
Summarizing the data!
Time taken: 9.496569871902466
Finished summarizing the data!
---
Time taken: 11.358048677444458
Finished summarizing the data!
---
Time taken: 15.699802160263062
Finished summarizing the data!
---


---

## Expert Interview Questions:


In [16]:
interview_chain = InterviewChain(topic=TOPIC, document_summaries=summaries)
interview_questions = interview_chain()

----------------------------------------

## Answer The Interview Questions:

In [17]:
for question in interview_questions.questions:
    print(f"Answer the following question: {question.question}\n", flush=True)
    answer = input(f"Answer the following question: {question.question}\n")
    print('------------------------------------------')
    question.answer = answer

Answer the following question: What is the history of neural networks and how have they evolved over time?

------------------------------------------
Answer the following question: What is the main difference between deep learning and traditional machine learning?

------------------------------------------
Answer the following question: How are neural networks structured and how do they process information?

------------------------------------------
Answer the following question: What are some practical applications of neural networks in the field of finance and investing?

------------------------------------------
Answer the following question: What are some criticisms or limitations of neural networks that researchers have identified?

------------------------------------------


---

## General Article Outline:


In [18]:
from article_outline_generation import BlogOutlineGenerator
blog_outline_generator = BlogOutlineGenerator(topic=TOPIC, questions_and_answers=[item.dict()  
                                                                                  for item in interview_questions.questions ] )
questions_and_answers = blog_outline_generator.questions_and_answers
outline_result = blog_outline_generator.generate_outline(summaries)

Generating the outline...
---
Finished generating the outline!
---


---

## Article Text Generation:


In [19]:
from article_generation import ContentGenerator
content_gen = ContentGenerator(topic=TOPIC, outline=outline_result, questions_and_answers=questions_and_answers)
content_gen.split_and_vectorize_documents(text_documents)
blog_post = content_gen.generate_blog_post()

Generating the blog post...
---
Finished generating the blog post!
---


In [20]:
blog_post

['# 1. History and Evolution of Neural Networks\n\nNeural networks have a rich history that spans over two centuries. The concept of using interconnected nodes to simulate the human brain\'s neural network has fascinated researchers for a long time. In this section, we will explore the key milestones in the history and evolution of neural networks.\n\n## Early Developments\n\n- In the early 1800s, mathematicians such as Legendre and Gauss pioneered the method of least squares or linear regression, which laid the foundation for neural networks.\n- In 1925, Wilhelm Lenz and Ernst Ising created the Ising model, an artificial recurrent neural network consisting of threshold elements.\n- In 1972, Shun\'ichi Amari made the Ising model adaptive, and John Hopfield popularized it in 1982.\n\n## McCulloch and Pitts: A Logical Calculus of Nervous Activity\n\nIn 1943, Warren McCulloch and Walter Pitts published "A Logical Calculus of the Ideas Immanent in Nervous Activity," which marked a signific

In [22]:
from image_generation_chain import create_image
image = create_image(outline_result.title)

---

## Title Tag Optimization:


In [23]:
%pip install thumb --quiet # This is a tool for evaluation of the potential blog post titles.

Note: you may need to restart the kernel to use updated packages.


In [25]:
import os
import thumb

# Set the environment variables (get your API key: https://platform.openai.com/account/api-keys)
# os.environ["LANGCHAIN_API_KEY"] = "YOUR_API_KEY_HERE" # optional: for langsmith tracing

In [26]:
base_prompt = f"Generate a title tag for this blog post on topic: {TOPIC}, Title: {outline_result.title}. The first 200 characters: {''.join(blog_post)[0:200]}"

# Set up a prompt templates for the a/b test:
prompt_a = base_prompt + ' ' + "Write the title in a style of {style}. Make sure that it is SEO friendly."
prompt_b = base_prompt + ' ' + "Write the title in a style of {style}. Make sure that is reader friendly. Focus on the reader."

# Set test cases with different input variables:
cases = [
  {"style": "business professional"},
  {"style": "b2b marketing"},
  ]

# Generate the responses
test = thumb.test([prompt_a, prompt_b], cases, runs=10)

--- Logging error ---
Traceback (most recent call last):
  File "/Users/jamesaphoenix/opt/anaconda3/lib/python3.9/site-packages/langchain/callbacks/manager.py", line 1763, in _configure
    handler = LangChainTracer(project_name=tracer_project)
  File "/Users/jamesaphoenix/opt/anaconda3/lib/python3.9/site-packages/langchain/callbacks/tracers/langchain.py", line 77, in __init__
    self.client = client or _get_client()
  File "/Users/jamesaphoenix/opt/anaconda3/lib/python3.9/site-packages/langchain/callbacks/tracers/langchain.py", line 45, in _get_client
    _CLIENT = Client()
  File "/Users/jamesaphoenix/opt/anaconda3/lib/python3.9/site-packages/langsmith/client.py", line 231, in __init__
    _validate_api_key_if_hosted(self.api_url, self.api_key)
  File "/Users/jamesaphoenix/opt/anaconda3/lib/python3.9/site-packages/langsmith/client.py", line 162, in _validate_api_key_if_hosted
    raise ls_utils.LangSmithUserError(
langsmith.utils.LangSmithUserError: API key must be provided when usi

HTML(value='"The Definitive Guide to Neural Networks: Uncovering the History and Evolution"')

IntProgress(value=0, description='Progress:', max=40)

Label(value='ThumbTest: 31f4e68e')